# DEMO CLIP

#### Author Jiacheng Huang

In this tutorial, we will cover the following basics for labeling and fine-tuning multimodal data using the CLIP module from LabelGenius (https://github.com/mediaccs/LabelGenius):

1. Labeling image data with CLIP
2. Labeling text data with CLIP
3. Combining text and image inputs for multimodal classification
4. Fine-tuning CLIP for improved task-specific performance


#### Cite LabelGenius
Huang, J., Zhang, Z., & Su, C. C. (2025). LabelGenius: A Tool for Dynamic and Flexible Labeling for LLM-based Multimodal Content Labeling [Computer software]. GitHub. https://github.com/mediaccs/LabelGenius

In [1]:
! pip install labelgenius

In [2]:
# Import the functions
import pandas as pd
from labelgenius import classification_CLIP_0_shot, finetune_CLIP, classification_CLIP_finetuned, auto_verification

Using device: cpu


In [3]:
# change directory to the folder contain the data
# import os
# os.chdir('/Users/jiacheng/Desktop/LLM/DEMO')

Demo 1: Single-Category Classification using N24News Dataset
-------------------------------------------------------------

This demo shows how to classify a single news article into one of 24 category
using the N24News dataset. Each article in the dataset includes both textual
and visual information.

Source: https://aclanthology.org/2022.lrec-1.729/


Each article contains the following fields:
- 'section': Ground truth label (one of 24 category)
- 'headline': Title of the article
- 'abstract': Short summary of the article
- 'article': Full text content
- 'article_url': Link to the original article
- 'image': Encoded image or metadata (optional)
- 'caption': Image caption
- 'image_id': Unique image identifier
- 'image_path': Path to the associated image (e.g., 'N24News/imgs_200_sample1/12345.jpg')
- 'article_id': Unique article identifier

Image file: Multimodal_image

Example category (See prompt_D1 for the complete category):
------------------------
1. Health
2. Science
3. Television
...
24. Global Business

Reference:
----------
Wang, Z., Shan, X., Zhang, X., & Yang, J. (2022).
N24News: A New Dataset for Multimodal News Classification.
In *Proceedings of the Thirteenth Language Resources and Evaluation Conference* (pp. 6768–6775). LREC.


### Demo 1a: Single-Category Text Classification

**Datasets:**
- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `D1_2.csv`: Used for testing the fine-tuned model's performance.


In [4]:
# Define the descriptive prompt for each category
prompt_D1_CLIP = [
    "a news article about health, including medical news, public health issues, fitness, mental health, and wellness.",
    "a news article about science, covering scientific discoveries, research studies, space exploration, and innovations.",
    "a news article about television, featuring TV shows, series reviews, industry news, and streaming platforms.",
    "a news article about travel, focusing on tourism, destinations, travel guides, airlines, and vacation trends.",
    "a news article about movies, including film industry news, reviews, box office reports, and upcoming releases.",
    "a news article about dance, covering ballet, contemporary styles, street dance, performances, and dance competitions.",
    "a news article about real estate, highlighting housing market trends, property sales, architecture, and urban planning.",
    "a news article about the economy, featuring macroeconomics, inflation, stock markets, GDP growth, and financial policies.",
    "a news article about sports, covering professional sports, competitions, athlete news, and game Demo_results.",
    "a news article about theater, featuring plays, Broadway shows, live performances, and stage production reviews.",
    "a news article about opinion pieces, including editorials, analysis, and expert commentaries.",
    "a news article about music, covering albums, artists, concerts, festivals, and industry trends.",
    "a news article about books, featuring literature, bestsellers, author interviews, and book reviews.",
    "a news article about art and design, showcasing fine arts, visual arts, museums, exhibitions, and design trends.",
    "a news article about style, including fashion trends, beauty, personal style, and cultural aesthetics.",
    "a news article about media, covering journalism, publishing, digital media, and mass communication.",
    "a news article about food, featuring restaurants, cooking, recipes, culinary trends, and food culture.",
    "a news article about well-being, focusing on lifestyle, personal development, mental well-being, and self-care.",
    "a news article about fashion, covering clothing, designers, fashion weeks, and industry insights.",
    "a news article about technology, featuring AI, gadgets, software, cybersecurity, and tech innovations.",
    "a news article about personal finance, including investing, budgeting, and financial planning.",
    "a news article about education, featuring schools, universities, learning methods, and education policies.",
    "a news article about automobiles, covering car industry news, electric vehicles, reviews, and trends.",
    "a news article about global business, featuring international trade, corporations, mergers, and global markets."
]

# Define the list of 24 category labels
category_D1_CLIP = [
    "1", "2", "3", "4", "5", "6",
    "7", "8", "9", "10", "11", "12",
    "13", "14", "15", "16", "17", "18",
    "19", "20", "21", "22", "23", "24"
]

## CLIP: local labeling

In [5]:
D1a_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    mode="text",
    prompt=prompt_D1_CLIP,
    text_column=["headline", "abstract"],
    predict_column="D1a_CLIP_inital_labeling",
)


D1a_CLIP_inital_labeling.to_csv("Demo_result/D1a_CLIP_inital_labeling.csv", index=False)

Loaded 20 records


Predicting: 100%|██████████| 20/20 [00:00<00:00, 58.90it/s]


### Let's compare the CLIP label with the ground-truth human label ("section_numeric")

In [6]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1a_CLIP_inital_labeling,
    predicted_cols="D1a_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
)


== Verification of 'D1a_CLIP_inital_labeling' vs. 'section_numeric' ==
Accuracy:   55.00%
Macro F1:   53.33%
Micro  F1:  55.00%

Full classification report:
              precision    recall  f1-score   support

           3       1.00      0.67      0.80         3
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
          10       1.00      1.00      1.00         1
          11       0.50      0.33      0.40         3
          13       1.00      0.67      0.80         3
          16       0.00      0.00      0.00         1
          19       1.00      0.50      0.67         2
          20       0.11      1.00      0.20         1
          23       1.00      1.00      1.00         2

    accuracy                           0.55        20
   macro avg       0.60      0.56      0.53        20
weighted avg       0.68      0.55      0.58        20


Confusion matrix:
[[2 0 0 0 

### Okay, the accuracy of the text-only data is 55%. Let's try to fine-tune it based on the true column "section_numeric".

#### Note. This demo only fine-tunes the first 20 records, so the result after fine-tuning could be worse. Usually, finetuning for CLIP works for 1000+ records.

### finetune: CLIP

In [7]:
# finetune CLIP
finetune_CLIP(
    mode="text",
    text_path="Demo_data/D1_1.csv",
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


Loaded 20 records
Number of classes: 24
Use text: True, Use image: False
Text columns: ['headline']
Label column: section_numeric
Training set class distribution: section_numeric
11    3
13    3
3     2
23    2
19    1
10    1
8     1
16    1
20    1
5     1
Name: count, dtype: int64
Validation set class distribution: section_numeric
4     1
5     1
3     1
19    1
Name: count, dtype: int64


Epoch 1/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 28.04it/s]


Epoch 1/20: Train Loss: 3.2003 | Train Acc: 0.00% | Val Loss: 3.1886 | Val Acc: 0.00%


Epoch 2/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 26.31it/s]


Epoch 2/20: Train Loss: 3.1605 | Train Acc: 6.25% | Val Loss: 3.1754 | Val Acc: 0.00%


...

Epoch 19/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 24.89it/s]


Epoch 19/20: Train Loss: 2.7613 | Train Acc: 100.00% | Val Loss: 3.1375 | Val Acc: 25.00%


Epoch 20/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 23.72it/s]

Epoch 20/20: Train Loss: 2.7550 | Train Acc: 100.00% | Val Loss: 3.1366 | Val Acc: 25.00%
Fine-tuning complete! Best validation accuracy: 25.00%


25.0

In [8]:
# Classify with a fine‑tuned model
D1a_CLIP_finetuned = classification_CLIP_finetuned(
    mode="text",
    text_path="Demo_data/D1_2.csv",
    text_column=["headline", "abstract"],
    model_name="Demo_finetuned_CLIP/D1a_CLIP_model_finetuned.pth",
    predict_column="D1a_CLIP_finetuned",
)


D1a_CLIP_finetuned.to_csv("Demo_result/D1a_CLIP_finetuned.csv", index=False)

20 pieces of data loaded


predicting: 100%|██████████| 3/3 [00:00<00:00, 12.22it/s]

Labeling completed!


In [9]:
## Check the accuracy: CLIP after finetune
auto_verification(
    D1a_CLIP_finetuned,
    predicted_cols="D1a_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)


== Verification of 'D1a_CLIP_finetuned' vs. 'section_numeric' ==
Accuracy:   0.00%
Macro F1:   0.00%
Micro  F1:  0.00%

Full classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       0.0
           7       0.00      0.00      0.00       1.0
           8       0.00      0.00      0.00       0.0
           9       0.00      0.00      0.00       1.0
          10       0.00      0.00      0.00       2.0
          11       0.00      0.00      0.00       0.0
          13       0.00      0.00      0.00       2.0
          14       0.00      0.00      0.00       1.0
          16       0.00      0.00      0.00       3.0
          17       0.00      0.00      0.00       2.0
          18       0.00      0.00      0.00       0.0
          19       0.00      0.00      0.00       1.0
          20       0.00      0.00      0

### Not surprisingly, the accuracy after fine-tuning is worse. 
### If you want to see how finetune improve performance with larger dataset, please look at the demo provided on our GitHub, which was fine-tuned on a large dataset.

# Demo 1b: Classify single-category image data

**Datasets:**

- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `imgs_40_1`: Used for initial labeling and fine-tuning.

- `D1_2.csv`: Used for testing the fine-tuned model's performance.
- `imgs_40_2`: Used for testing the fine-tuned model's performance.


### CSV File Requirements

Each CSV file must include at minimum:

- `image_id`  
  – The base filename (without extension) of each image in the corresponding folder.

## CLIP: local labeling

In [10]:
D1b_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    img_dir="Demo_data/D1_imgs_1",
    mode="image",
    prompt=prompt_D1_CLIP,
    predict_column="D1b_CLIP_inital_labeling",
)


D1b_CLIP_inital_labeling.to_csv("Demo_result/D1b_CLIP_inital_labeling.csv", index=False)

Loaded 20 records


Predicting: 100%|██████████| 20/20 [00:01<00:00, 17.11it/s]


In [11]:
D1b_CLIP_inital_labeling

,section,headline,article_url,article,abstract,article_id,image,caption,image_id,image_path,section_numeric,D1b_CLIP_inital_labeling
0,Fashion & Style,"On This Runway, Non-Models and Cool Kids",https://www.nytimes.com/2016/09/10/fashion/eck...,"Over Labor Day weekend, a steady stream of hop...",How the Eckhaus Latta designers select fashion...,e9cd6477-5eb6-58b2-8e33-fd2d881bf656,https://static01.nyt.com/images/2016/09/10/fas...,"Mike Eckhaus, center right, and Zoe Latta, cen...",e9cd6477-5eb6-58b2-8e33-fd2d881bf656,/Demo_data/D1_imgs_1/e9cd6477-5eb6-58b2-8e33-f...,19,11
1,Theater,"The New 42nd Street, a Theater Nonprofit, Name...",https://www.nytimes.com/2019/06/11/theater/new...,The New 42nd Street -- the nonprofit organizat...,Russell Granet will be the new president and c...,acdcd1ef-71a9-55f2-9993-7a231d57396f,https://static01.nyt.com/images/2019/06/11/art...,"The New 42nd Street board chairwoman, Fiona Ru...",acdcd1ef-71a9-55f2-9993-7a231d57396f,/Demo_data/D1_imgs_1/acdcd1ef-71a9-55f2-9993-7...,10,21
2,Economy,Tax Tactics Threaten Public Funds,https://www.nytimes.com/2014/10/02/business/ec...,When the European Commission charged this week...,If global corporations can continue to evade t...,1de5c156-9cea-5971-9fdf-b6a4ce9bf35c,https://static01.nyt.com/images/2012/02/29/bus...,Eduardo Porter,1de5c156-9cea-5971-9fdf-b6a4ce9bf35c,/Demo_data/D1_imgs_1/1de5c156-9cea-5971-9fdf-b...,8,13
3,Television,Review: 'The Chi' Returns to the South Side of...,https://www.nytimes.com/2019/04/05/arts/televi...,"""The Chi"" was built around a series of shootin...",Lena Waithe's neighborhood drama on Showtime d...,7bb11a9c-2f6d-57f9-bb36-db7f00f59589,https://static01.nyt.com/images/2019/04/05/art...,"Alex Hibbert as Kevin in ""The Chi,"" a sprawlin...",7bb11a9c-2f6d-57f9-bb36-db7f00f59589,/Demo_data/D1_imgs_1/7bb11a9c-2f6d-57f9-bb36-d...,3,3
4,Opinion,Banks Should Face History and Pay Reparations,https://www.nytimes.com/2020/06/26/opinion/sun...,Ms. Blackwell is founder in residence at Polic...,The financial industry can close the wealth ga...,165426ae-1e7c-5193-8596-ef96357645a8,https://static01.nyt.com/images/2020/06/25/opi...,A Wells Fargo bank in Minneapolis was set on f...,165426ae-1e7c-5193-8596-ef96357645a8,/Demo_data/D1_imgs_1/165426ae-1e7c-5193-8596-e...,11,12
5,Books,"Mercedes Barcha, Gabriel García Márquez's Wife...",https://www.nytimes.com/2020/08/23/books/merce...,"Mercedes Barcha and her husband, the Colombian...","For nearly 60 years, she was the Nobel Prize-w...",23348aa1-6fa8-5ca8-a875-71fe704e6dd2,https://static01.nyt.com/images/2020/08/24/obi...,"Mercedes Barcha and her husband, the Colombian...",23348aa1-6fa8-5ca8-a875-71fe704e6dd2,/Demo_data/D1_imgs_1/23348aa1-6fa8-5ca8-a875-7...,13,10
6,Opinion,The George Floyd Election,https://www.nytimes.com/2020/06/03/opinion/geo...,"Mike Erlandson, a former chairman of the state...",How the protests come to be viewed may determi...,32d42c23-4bd8-504a-8151-912b4c3f002d,https://static01.nyt.com/images/2020/06/03/opi...,Protests over the killing of George Floyd in N...,32d42c23-4bd8-504a-8151-912b4c3f002d,/Demo_data/D1_imgs_1/32d42c23-4bd8-504a-8151-9...,11,11
7,Books,"For Black and Mixed-Race Women, Hair and Ident...",https://www.nytimes.com/2020/03/17/books/revie...,What do we lose in translation? How do we know...,"In ""That Hair,"" the Portuguese writer Djaimili...",7e7703aa-4180-57a3-9781-c74a25d31afb,https://static01.nyt.com/images/2020/03/22/boo...,Djaimilia Pereira de Almeida,7e7703aa-4180-57a3-9781-c74a25d31afb,/Demo_data/D1_imgs_1/7e7703aa-4180-57a3-9781-c...,13,13
8,Books,A Contemporary Feminist Spin on the Traditiona...,https://www.nytimes.com/2019/07/08/books/revie...,Motivated in part by guilt as a privileged out...,"In Madeline ffitch's debut, ""Stay and Fight,"" ...",c030fbee-2601-54f5-a9df-75fa8c13368c,https://static01.nyt.com/images/2019/06/27/boo...,"Madeline ffitch's debut novel, ""Stay and Fight...",c030fbee-2601-54f5-a9df-75fa8c13368c,/Demo_data/D1_imgs_1/c030fbee

In [12]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1b_CLIP_inital_labeling,
    predicted_cols="D1b_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
);


== Verification of 'D1b_CLIP_inital_labeling' vs. 'section_numeric' ==
Accuracy:   35.00%
Macro F1:   23.59%
Micro  F1:  35.00%

Full classification report:
              precision    recall  f1-score   support

           3       0.50      0.33      0.40         3
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       0.22      0.67      0.33         3
          12       0.00      0.00      0.00         0
          13       0.67      0.67      0.67         3
          16       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         2
          20       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         0
          23       1.00      0.50      0.67         2

    accuracy                           0.35        20
   macro avg       0.26      0

### finetune: CLIP

In [13]:
# finetune CLIP
finetune_CLIP(
    mode="text",
    text_path="Demo_data/D1_1.csv",
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1b_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


Loaded 20 records
Number of classes: 24
Use text: True, Use image: False
Text columns: ['headline']
Label column: section_numeric
Training set class distribution: section_numeric
11    3
13    3
3     2
23    2
19    1
10    1
8     1
16    1
20    1
5     1
Name: count, dtype: int64
Validation set class distribution: section_numeric
4     1
5     1
3     1
19    1
Name: count, dtype: int64


Epoch 1/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 11.20it/s]


Epoch 1/20: Train Loss: 3.1734 | Train Acc: 0.00% | Val Loss: 3.1904 | Val Acc: 0.00%


Epoch 2/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 12.58it/s]


Epoch 2/20: Train Loss: 3.1322 | Train Acc: 50.00% | Val Loss: 3.1811 | Val Acc: 0.00%


...

Epoch 19/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 23.79it/s]


Epoch 19/20: Train Loss: 2.7537 | Train Acc: 100.00% | Val Loss: 3.1263 | Val Acc: 25.00%


Epoch 20/20 [Val]: 100%|██████████| 1/1 [00:00<00:00, 24.25it/s]

Epoch 20/20: Train Loss: 2.7479 | Train Acc: 100.00% | Val Loss: 3.1259 | Val Acc: 25.00%
Fine-tuning complete! Best validation accuracy: 25.00%


25.0

In [14]:
# Classify with a fine‑tuned model
D1b_CLIP_finetuned = classification_CLIP_finetuned(
    mode="image",
    text_path="Demo_data/D1_2.csv",
    img_dir="Demo_data/D1_imgs_2",
    model_name="Demo_finetuned_CLIP/D1b_CLIP_model_finetuned.pth",
    predict_column="D1b_CLIP_finetuned",
)

D1b_CLIP_finetuned.to_csv("Demo_result/D1b_CLIP_finetuned.csv", index=False)

20 pieces of data loaded


predicting: 100%|██████████| 3/3 [00:00<00:00,  6.14it/s]

Labeling completed!


In [15]:
## Check the accuracy: CLIP after finetune
# Merge the Demo_result of the image with the human label
auto_verification(
    D1b_CLIP_finetuned,
    predicted_cols="D1b_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)


== Verification of 'D1b_CLIP_finetuned' vs. 'section_numeric' ==
Accuracy:   5.00%
Macro F1:   1.67%
Micro  F1:  5.00%

Full classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         3
          17       0.00      0.00      0.00         2
          19       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         2
          21       0.14      1.00      0.25         1
          23       0.00      0.00      0

# Demo 1c: Classify single-category text + image data

**Datasets:**
- `D1_1.csv`: Used for initial labeling and fine-tuning.
- `D1_imgs_1`: Used for initial labeling and fine-tuning.

- `D1_2.csv`: Used for testing the fine-tuned model's performance.
- `D1_imgs_2`: Used for testing the fine-tuned model's performance.


The text dataset should contain a column `image_path` to map the images for each row.

## CLIP: local labeling

In [16]:
D1c_CLIP_inital_labeling = classification_CLIP_0_shot(
    text_path="Demo_data/D1_1.csv",
    img_dir="Demo_data/D1_imgs_1",
    mode="both",
    prompt=prompt_D1_CLIP,
    text_column=["headline", "abstract"],
    predict_column="D1c_CLIP_inital_labeling",
)

D1c_CLIP_inital_labeling.to_csv("Demo_result/D1c_CLIP_inital_labeling.csv", index=False)

Loaded 20 records


Predicting: 100%|██████████| 20/20 [00:01<00:00, 13.39it/s]


In [17]:
## Check the accuracy: CLIP initial labeling
auto_verification(
    D1c_CLIP_inital_labeling,
    predicted_cols="D1c_CLIP_inital_labeling",
    true_cols="section_numeric",
    category=category_D1_CLIP
);


== Verification of 'D1c_CLIP_inital_labeling' vs. 'section_numeric' ==
Accuracy:   65.00%
Macro F1:   49.37%
Micro  F1:  65.00%

Full classification report:
              precision    recall  f1-score   support

           3       0.75      1.00      0.86         3
           4       1.00      1.00      1.00         1
           5       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
          10       0.33      1.00      0.50         1
          11       0.50      0.33      0.40         3
          12       0.00      0.00      0.00         0
          13       1.00      1.00      1.00         3
          16       0.00      0.00      0.00         1
          19       1.00      0.50      0.67         2
          20       0.33      1.00      0.50         1
          23       1.00      1.00      1.00         2

    accuracy                           0.65        20
   macro avg       0.49      0.57      0.49        20
weighted avg       0.62      0

### finetune: CLIP

In [18]:
# finetune CLIP
finetune_CLIP(
    mode="both",
    text_path="Demo_data/D1_1.csv",
    text_column=["headline", "abstract"],
    img_dir="Demo_data/D1_imgs_1",
    true_label="section_numeric",
    model_name="Demo_finetuned_CLIP/D1c_CLIP_model_finetuned.pth",
    num_epochs=20,
    batch_size=8,
    learning_rate=1e-5,
)


Loaded 20 records
Number of classes: 24
Use text: True, Use image: True
Text columns: ['headline', 'abstract']
Label column: section_numeric
Training set class distribution: section_numeric
11    3
13    3
3     2
23    2
19    1
10    1
8     1
16    1
20    1
5     1
Name: count, dtype: int64
Validation set class distribution: section_numeric
4     1
5     1
3     1
19    1
Name: count, dtype: int64


Epoch 1/20 [Val]: 100%|██████████| 1/1 [00:00<00:00,  9.25it/s]


Epoch 1/20: Train Loss: 3.1802 | Train Acc: 0.00% | Val Loss: 3.1483 | Val Acc: 25.00%
Model saved! Best validation accuracy: 25.00%


Epoch 2/20 [Val]: 100%|██████████| 1/1 [00:00<00:00,  9.01it/s]


Epoch 2/20: Train Loss: 3.1163 | Train Acc: 43.75% | Val Loss: 3.1450 | Val Acc: 25.00%


...

Epoch 19/20 [Val]: 100%|██████████| 1/1 [00:00<00:00,  8.20it/s]


Epoch 19/20: Train Loss: 2.6789 | Train Acc: 100.00% | Val Loss: 3.1329 | Val Acc: 0.00%


Epoch 20/20 [Val]: 100%|██████████| 1/1 [00:00<00:00,  7.81it/s]

Epoch 20/20: Train Loss: 2.6791 | Train Acc: 100.00% | Val Loss: 3.1314 | Val Acc: 0.00%
Fine-tuning complete! Best validation accuracy: 25.00%


25.0

In [19]:
# Classify with a fine‑tuned model
D1c_CLIP_finetuned = classification_CLIP_finetuned(
    mode="both",
    text_path="Demo_data/D1_2.csv",
    text_column=["headline", "abstract"],
    img_dir="Demo_data/D1_imgs_2",
    model_name="Demo_finetuned_CLIP/D1c_CLIP_model_finetuned.pth",
    predict_column="D1c_CLIP_finetuned",
    
)
D1c_CLIP_finetuned.to_csv("Demo_result/D1c_CLIP_finetuned.csv", index=False)

20 pieces of data loaded


predicting: 100%|██████████| 3/3 [00:00<00:00,  4.38it/s]


Labeling completed!


In [20]:
D1c_CLIP_finetuned

,section,headline,article_url,article,abstract,article_id,image,caption,image_id,image_path,section_numeric,D1c_CLIP_finetuned
0,Technology,The Sun Sets. The Wind Dies. But Energy Data I...,https://www.nytimes.com/2018/09/26/technology/...,How do New York Times journalists use technolo...,Alternative power sources have upended the ele...,5631bfd7-67ff-5fa3-8efd-37d65f3e234e,https://static01.nyt.com/images/2018/09/27/bus...,"Ivan Penn, who reports on alternative energy, ...",5631bfd7-67ff-5fa3-8efd-37d65f3e234e,/Demo_data/D1_imgs_2/5631bfd7-67ff-5fa3-8efd-3...,20,2
1,Automobiles,Monday Motorsports: Ed Carpenter Wins Another ...,https://www.nytimes.com/2014/05/20/automobiles...,Ed Carpenter hasn't run an IndyCar race since ...,"Carpenter collected a $100,000 prize for his s...",c55a5863-a87d-5470-8210-055e307b8956,https://static01.nyt.com/images/2014/05/20/aut...,Ed Carpenter won pole position Sunday for the ...,c55a5863-a87d-5470-8210-055e307b8956,/Demo_data/D1_imgs_2/c55a5863-a87d-5470-8210-0...,23,2
2,Real Estate,Recent Commercial Real Estate Transactions,https://www.nytimes.com/2019/01/15/realestate/...,497 Atlantic Avenue (between Nevins Street and...,Recent commercial real estate transactions in ...,f3f0dc8e-3241-5e5e-a0d9-b9a3bfc7862f,https://static01.nyt.com/images/2019/01/16/rea...,A four-story walk-up is on the market for $2.5...,f3f0dc8e-3241-5e5e-a0d9-b9a3bfc7862f,/Demo_data/D1_imgs_2/f3f0dc8e-3241-5e5e-a0d9-b...,7,12
3,Television,The Best Movies and TV Shows Coming to Netflix...,https://www.nytimes.com/2019/12/01/arts/televi...,Watching is The New York Times's TV and film r...,"Every month, subscription streaming services a...",f0ae30f1-09b9-5334-a32c-04cf793a2aef,https://static01.nyt.com/images/2019/12/01/art...,"Clockwise from top left: Scenes from ""Moonligh...",f0ae30f1-09b9-5334-a32c-04cf793a2aef,/Demo_data/D1_imgs_2/f0ae30f1-09b9-5334-a32c-0...,3,2
4,Theater,"Review: In 'Leopoldstadt,' Tom Stoppard Reckon...",https://www.nytimes.com/2020/02/12/theater/leo...,LONDON -- Do you remember? Don't you remember?...,In what he says will probably be his last work...,da8070be-edc1-5d6f-a78c-3962249598cc,https://static01.nyt.com/images/2020/02/12/art...,"Adrian Scarborough, left, and Luke Thallon in ...",da8070be-edc1-5d6f-a78c-3962249598cc,/Demo_data/D1_imgs_2/da8070be-edc1-5d6f-a78c-3...,10,2
5,Food,"A Savory, Tender Gnocchi Poised for Success",https://www.nytimes.com/2017/02/03/dining/rico...,"Gnocchi, those savory little dumplings, are fr...",Beginner's attempts at potato gnocchi usually ...,c775b7f0-d7a6-5e66-a249-8ea0da026e35,https://static01.nyt.com/images/2017/02/08/din...,"Ricotta gnocchi with parsley pesto, topped wit...",c775b7f0-d7a6-5e66-a249-8ea0da026e35,/Demo_data/D1_imgs_2/c775b7f0-d7a6-5e66-a249-8...,17,4
6,Media,Donald Trump Elicits Shock and Biting Satire i...,https://www.nytimes.com/2016/02/29/business/me...,LONDON -- He has been depicted as a snarling d...,Once dismissive of Mr. Trump's presidential ca...,ed504d2e-939b-5641-b3a8-75175bc7efc3,https://static01.nyt.com/images/2016/02/29/bus...,"On a recent issue of Der Spiegel, the headline...",ed504d2e-939b-5641-b3a8-75175bc7efc3,/Demo_data/D1_imgs_2/ed504d2e-939b-5641-b3a8-7...,16,3
7,Technology,"Apple, Google and a Deal That Controls the Int...",https://www.nytimes.com/2020/10/25/technology/...,"OAKLAND, Calif. -- When Tim Cook and Sundar Pi...","In a landmark antitrust complaint, the Justice...",da1996b0-46fc-5d47-b06f-8c0be9deaa95,https://static01.nyt.com/images/2020/10/25/bus...,"In 2007, Steve Jobs, left, Apple's co-founder,...",da1996b0-46fc-5d47-b06f-8c0be9deaa95,/Demo_data/D1_imgs_2/da1996b0-46fc-5d47-b06f-8...,20,2
8,Media,"Google Buying Apple? Sorry, That Was Just a Test",https://www.nytimes.com/2017/10/10/business/me...,The Dow Jones Newswire headline seemed too far...,A far-fetched headline and short story that we...,383097ce-2c44-59b9-87be-cdbe235de68d,https://static01.nyt.com/images/2017/10/15/bus...,A bogus headline on the Dow Jones News

In [21]:
## Check the accuracy: CLIP after finetune
auto_verification(
    D1c_CLIP_finetuned,
    predicted_cols="D1c_CLIP_finetuned",
    true_cols="section_numeric",
    category=category_D1_CLIP
)


== Verification of 'D1c_CLIP_finetuned' vs. 'section_numeric' ==
Accuracy:   0.00%
Macro F1:   0.00%
Micro  F1:  0.00%

Full classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       0.0
           5       0.00      0.00      0.00       0.0
           6       0.00      0.00      0.00       0.0
           7       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       1.0
          10       0.00      0.00      0.00       2.0
          12       0.00      0.00      0.00       0.0
          13       0.00      0.00      0.00       2.0
          14       0.00      0.00      0.00       1.0
          16       0.00      0.00      0.00       3.0
          17       0.00      0.00      0.00       2.0
          19       0.00      0.00      0.00       1.0
          20       0.00      0.00      0